In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier

In [ ]:
pd.set_option('display.max_rows', None)

# Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv', sep=',')
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv', sep=',')
test= pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv', sep=',')

### Features Extraction

In [ ]:
train = train.set_index('PassengerId')

#df with train + test data
dfg = pd.concat([train, test], axis=0)

### 1.Age

In [ ]:
train['Age'] = train['Age'].replace(np.nan, dfg['Age'].median())
train['Age'] = round(train['Age'],0)

bins = ['Y1', 'Y2', 'Y3', 'Y4', 'M1', 'M2', 'E']
train['Age_Bin'] = pd.cut(x=train['Age'],
                            bins=[0, 5 , 10, 15, 20, 30, 50,1000],
                            labels=bins,right=False)
train['Age_Bin'] = train['Age_Bin'].astype('str')
train['Age_Bin'] = train['Age_Bin']+train['Sex']
df_Age_bin = pd.get_dummies(train['Age_Bin'], prefix='Age_bin')

### 2.Fare

In [ ]:
dfg['Age_Bin'] = pd.cut(x=dfg['Age'],
                            bins=[0, 5 , 10, 15, 20, 30, 50,1000],
                            labels=bins,right=False)
dfg.groupby(by=['Sex','Pclass'])['Fare'].median()

In [ ]:
train.loc[(train['Fare'].isna()) & (train['Pclass']==1) & (train['Sex']=='female'), 'Fare']=85.40
train.loc[(train['Fare'].isna()) & (train['Pclass']==2) & (train['Sex']=='female'), 'Fare']=24.75
train.loc[(train['Fare'].isna()) & (train['Pclass']==3) & (train['Sex']=='female'), 'Fare']=12.54

train.loc[(train['Fare'].isna()) & (train['Pclass']==1) & (train['Sex']=='male'), 'Fare']=64.51
train.loc[(train['Fare'].isna()) & (train['Pclass']==2) & (train['Sex']=='male'), 'Fare']=14.23
train.loc[(train['Fare'].isna()) & (train['Pclass']==3) & (train['Sex']=='male'), 'Fare']=11.02

In [ ]:
#Fare bins
bins2 = ['L1', 'L2', 'L3', 'L4']
train['Fare_Bin'] = pd.cut(x=train['Fare'],
                            bins=[0,11, 30 , 60, 10000],
                            labels=bins2,right=False)
train['Fare_Bin'] = train['Fare_Bin'].astype('str')
df_Fare_bin = pd.get_dummies(train['Fare_Bin'], prefix='Fare_bin')

### 3.Cabin

In [ ]:
dfg['Cabin'] =dfg['Cabin'].str[0]
train['Cabin'] =train['Cabin'].str[0]
train['Cabin'] = train['Cabin'].fillna('Z')
train.loc[(train['Cabin']=='T'), 'Cabin']='Z'
df_cabin = pd.get_dummies(train['Cabin'], prefix='Cabin')

### 4.Embarked 

In [ ]:
#Used the most frequent caracter
train['Embarked'] = train['Embarked'].fillna('S')
df_embarked = pd.get_dummies(train['Embarked'], prefix='Embark')

### 5.Ticket 

In [ ]:
train['Ticket'] = train['Ticket'].str.replace('[^a-zA-Z]', '').str[:1]
train['Ticket'] = train['Ticket'].str.strip()
train['Ticket'] = train['Ticket'].fillna('ZZ')
train.loc[train['Ticket']=='', 'Ticket']='ZZ'
train.loc[train['Ticket']=='L', 'Ticket']='ZZ'
df_tiket = pd.get_dummies(train['Ticket'], prefix='ticket')

### 6.Name 

In [ ]:
df_name = pd.concat([train['Name'], test['Name']], axis=0)
df_name = pd.DataFrame(df_name, columns=['Name'])

df_name['FirstName'] = df_name['Name'].apply(lambda x:x.split(', ')[0])
df_name['SecondName'] = df_name['Name'].str.split(', ', 1, expand=True)[1]

le = LabelEncoder()
le1 = LabelEncoder()
df_name['FirstName'] = le.fit_transform(df_name['FirstName'])
df_name['SecondName'] = le1.fit_transform(df_name['SecondName'])

train['FirstName'] = train['Name'].apply(lambda x:x.split(', ')[0])
train['SecondName'] = train['Name'].str.split(', ', 1, expand=True)[1]

train['FirstName'] = le.transform(train['FirstName'])
train['SecondName'] = le1.transform(train['SecondName'])

### 7.Sex 

In [ ]:
train['Sex'] = train['Sex'].apply(lambda x: 1 if x=='female' else 0)

### 8.Pclass 

In [ ]:
train['Pclass'] = train['Pclass'].astype('str')
df_pclass = pd.get_dummies(train['Pclass'], prefix='class')

### 9.Family Size 

Get the Fatures related to Family size. An idea taken from: https://medium.datadriveninvestor.com/start-with-kaggle-a-comprehensive-guide-to-solve-the-titanic-challenge-8ac5815b0473

In [ ]:
# introducing a new feature : the size of families (including the passenger)
train['FamilySize'] = train['Parch'] + train['SibSp'] + 1

# introducing other features based on the family size
train['Singleton'] = train['FamilySize'].map(lambda s: 1 if s == 1 else 0)
train['SmallFamily'] = train['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
train['LargeFamily'] = train['FamilySize'].map(lambda s: 1 if 5 <= s else 0)

## EDA Features importance

In [ ]:
df = pd.concat([train['Fare'], train['Age'],train['SibSp'],train['Parch'],train['FamilySize'], train['Singleton'], train['SmallFamily'], train['LargeFamily'],train['Sex'], df_cabin,df_tiket, df_pclass, df_embarked ,train['FirstName'],train['SecondName'],df_Age_bin,df_Fare_bin,train['Survived']], axis=1)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data=df.corr())

In [ ]:
df = df.drop(columns='Survived')

### Adding 3 Kmeans Features

In [ ]:
km = KMeans(n_clusters=3, random_state=22, n_init=20)
df_km = km.fit_predict(df)
df_km = pd.DataFrame(df_km, index=df.index)
df_km = df_km.astype('str')
df_km = pd.get_dummies(df_km)

In [ ]:
df = pd.concat([df, df_km], axis=1)
df_target = train['Survived']

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_features='sqrt')
clf = clf.fit(df, df_target)

In [ ]:
features = pd.DataFrame()
features['feature'] = df.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(25, 25))

In [ ]:
df.columns

In [ ]:
df_cb = pd.concat([train['Fare'],train['SibSp'],train['Parch'], train['Age'],train['FamilySize'], train['Singleton'], train['SmallFamily'], train['LargeFamily'], train['Sex'], train['Cabin'],train['Ticket'], train['Pclass'], train['Embarked'] ,train['FirstName'], train['SecondName'],train['Age_Bin'],train['Fare_Bin'], df_km], axis=1)

## Optuna with Catboost

In [ ]:
#Category Columns
cat_cols = ['FamilySize', 'Singleton', 'SmallFamily', 'LargeFamily',
       'Sex', 'Cabin', 'Ticket', 'Pclass', 'Embarked', 'FirstName', 'SecondName',
       '0_0', '0_1', '0_2', 'Age_Bin', 'Fare_Bin']

In [ ]:
df_cb.columns

In [ ]:
import optuna

In [ ]:
def objective(trial , data = df_cb , target = df_target):
    train_x , test_x , train_y , test_y = train_test_split(data , target , \
            test_size = 0.028059109276941666 , random_state = 2)

    params = {'iterations':10000,
              'depth': trial.suggest_int("depth", 4, 70),
              'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 0.0001, 25, log=True),
              'bagging_temperature': trial.suggest_float("bagging_temperature", 0, 100),
              'auto_class_weights':trial.suggest_categorical('auto_class_weights', [None,'Balanced','SqrtBalanced']),
              'grow_policy': 'Lossguide',
              'loss_function':'Logloss',
              'bootstrap_type':trial.suggest_categorical("bootstrap_type", ['Poisson']),
              'use_best_model':True,
              'task_type':'GPU', 
              'cat_features':cat_cols,
              'eval_metric': 'Logloss',
              'learning_rate': trial.suggest_uniform('learning_rate' , 1e-5 , 1.0),
              #'max_bin': trial.suggest_int('max_bin', 5, 500),
              'verbose':False,
              'border_count':trial.suggest_int('max_bin', 5, 600),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 600),
              'subsample': trial.suggest_uniform('subsample' , 1e-5 , 1.0)
             }
    model = CatBoostClassifier(**params)
    model.fit(train_x , train_y , eval_set = [(test_x , test_y)] , early_stopping_rounds = 2000 , \
             verbose = False)
    preds = model.predict(test_x)
    acc = accuracy_score(test_y , preds)
    return acc

In [ ]:
study = optuna.create_study(direction = 'maximize' , study_name = 'cb')
study.optimize(objective , n_trials = 1)
print('numbers of the finished trials:' , len(study.trials))
print('the best params:' , study.best_trial.params)
print('the best value:' , study.best_value)

In [ ]:
#the best value: 0.7822523164647185
params={'depth': 38, 'l2_leaf_reg': 0.0010021036002324036, 'bagging_temperature': 54.72815390606476, 'auto_class_weights': None, 'bootstrap_type': 'Poisson', 'learning_rate': 0.0863109888471924, 'max_bin': 192, 'min_data_in_leaf': 338, 'subsample': 0.5399710139554038}

In [ ]:
test = test.set_index('PassengerId')

#Age
test['Age'] = test['Age'].replace(np.nan, dfg['Age'].median())
test['Age_Bin'] = pd.cut(x=test['Age'],
                            bins=[0, 5 , 10, 15, 20, 30, 50,1000],
                            labels=bins,right=False)
test['Age_Bin'] = test['Age_Bin'].astype('str')
test['Age_Bin'] = test['Age_Bin']+test['Sex']
dft_Age_bin = pd.get_dummies(test['Age_Bin'], prefix='Age_bin')


#Fare
test.loc[(test['Fare'].isna()) & (test['Pclass']==1) & (test['Sex']=='female'), 'Fare']=85.40
test.loc[(test['Fare'].isna()) & (test['Pclass']==2) & (test['Sex']=='female'), 'Fare']=24.75
test.loc[(test['Fare'].isna()) & (test['Pclass']==3) & (test['Sex']=='female'), 'Fare']=12.54

test.loc[(test['Fare'].isna()) & (test['Pclass']==1) & (test['Sex']=='male'), 'Fare']=64.51
test.loc[(test['Fare'].isna()) & (test['Pclass']==2) & (test['Sex']=='male'), 'Fare']=14.23
test.loc[(test['Fare'].isna()) & (test['Pclass']==3) & (test['Sex']=='male'), 'Fare']=11.02

test['Fare_Bin'] = pd.cut(x=test['Fare'],
                            bins=[0,11, 30 , 60, 10000],
                            labels=bins2,right=False)

test['Fare_Bin'] = test['Fare_Bin'].astype('str')
dft_Fare_bin = pd.get_dummies(test['Fare_Bin'], prefix='Fare_bin')


#Cabin
test['Cabin'] =test['Cabin'].str[0]
test['Cabin'] = test['Cabin'].fillna('Z')
test.loc[(test['Cabin']=='T'), 'Cabin']='Z'
dft_cabin = pd.get_dummies(test['Cabin'], prefix='Cabin')

#Embarked
test['Embarked'] = test['Embarked'].fillna('S')
dft_embarked = pd.get_dummies(test['Embarked'], prefix='Embark')

#Ticket
test['Ticket'] = test['Ticket'].str.replace('[^a-zA-Z]', '').str[:1]
test['Ticket'] = test['Ticket'].str.strip()
test['Ticket'] = test['Ticket'].fillna('ZZ')
test.loc[test['Ticket']=='', 'Ticket']='ZZ'
test.loc[test['Ticket']=='L', 'Ticket']='ZZ'
dft_tiket = pd.get_dummies(test['Ticket'], prefix='ticket')

#Name
test['FirstName'] = test['Name'].apply(lambda x:x.split(', ')[0])
test['SecondName'] = test['Name'].str.split(', ', 1, expand=True)[1]

test['FirstName'] = le.transform(test['FirstName'])
test['SecondName'] = le1.transform(test['SecondName'])

#Sex
test['Sex'] = test['Sex'].apply(lambda x: 1 if x=='female' else 0)

#Pclass
test['Pclass'] = test['Pclass'].astype('str')
dft_pclass = pd.get_dummies(test['Pclass'], prefix='class')

#Family Size
test['FamilySize'] = test['Parch'] + test['SibSp'] + 1

test['Singleton'] = test['FamilySize'].map(lambda s: 1 if s == 1 else 0)
test['SmallFamily'] = test['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
test['LargeFamily'] = test['FamilySize'].map(lambda s: 1 if 5 <= s else 0)

In [ ]:
dft = pd.concat([test['Fare'],test['SibSp'],test['Parch'], test['Age'],test['FamilySize'], test['Singleton'], test['SmallFamily'], test['LargeFamily'], test['Sex'],dft_cabin,dft_tiket, dft_pclass, dft_embarked, test['FirstName'],test['SecondName'], dft_Age_bin, dft_Fare_bin], axis=1)

In [ ]:
dft_km = km.predict(dft)
dft_km = pd.DataFrame(dft_km, index=dft.index)
dft_km = dft_km.astype('str')
dft_km = pd.get_dummies(dft_km)

In [ ]:
dft = pd.concat([dft, dft_km], axis=1)

In [ ]:
dft_cb = pd.concat([test['Fare'], test['SibSp'], test['Parch'],test['Age'],test['FamilySize'], test['Singleton'], test['SmallFamily'], test['LargeFamily'], test['Sex'], test['Cabin'],test['Ticket'], test['Pclass'], test['Embarked'] ,test['FirstName'],test['SecondName'],test['Age_Bin'],test['Fare_Bin'], dft_km], axis=1)

In [ ]:
params_cb = {
    'bootstrap_type': 'Poisson',
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'random_seed': 22,
    'task_type': 'GPU',
    'grow_policy': 'Lossguide',
    'depth': 38,
    'bagging_temperature': 54.72815390606476,
    'auto_class_weights': None,
    'cat_features': cat_cols,
    'learning_rate': 0.0863109888471924,
    'iterations':10000,
    'max_bin': 192,
    'min_data_in_leaf': 338,
    'subsample': 0.5399710139554038,
    'l2_leaf_reg': 0.0010021036002324036,
}

In [ ]:
preds3 = np.zeros(dft_cb.shape[0])
oof_preds3 = np.zeros(df_cb.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 22 , shuffle = True)
roc3 = []
n = 0
for trn_idx , val_idx in kf.split(df_cb , df_target):
    train_x = df_cb.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df_cb.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model3 = CatBoostClassifier(**params_cb)
    model3.fit(train_x , train_y , eval_set = [(val_x , val_y)], early_stopping_rounds = 3000 , verbose = False)
    preds3 += model3.predict_proba(dft_cb)[:,1]/kf.n_splits
    oof_preds3 += model3.predict_proba(df_cb)[:,1]/kf.n_splits
    roc3.append(accuracy_score(val_y , model3.predict(val_x)))
    fpr, tpr, thresholds = roc_curve(val_y , model3.predict_proba(val_x)[:,1])
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print(n+1 , roc3[n], 'Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
    n+=1

### Looking for the Best Threshold

In [ ]:
thresholds = np.arange(0, 1, 0.001)
# apply threshold to positive probabilities to create labels

def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

In [ ]:
scores = [accuracy_score(df_target, to_labels(oof_preds3, t)) for t in thresholds]

In [ ]:
ix = np.argmax(scores)
print('Threshold=%.3f, accuracy_Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
sub_sample['Survived'] = preds3

In [ ]:
#simple threshold
sub_sample['Survived'] = sub_sample['Survived'].apply(lambda x: 1 if x>thresholds[ix] else 0)

### Pseudo-Labels with the same model

In [ ]:
y = pd.concat([df_target, sub_sample['Survived']], axis=0)
X = pd.concat([df_cb, dft_cb], axis=0)
print(X.shape,y.shape)

In [ ]:
preds3 = np.zeros(dft_cb.shape[0])
oof_preds3 = np.zeros(X.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 22 , shuffle = True)
roc3 = []
n = 0
for trn_idx , val_idx in kf.split(X , y):
    train_x = X.iloc[trn_idx]
    train_y = y.iloc[trn_idx]
    val_x = X.iloc[val_idx]
    val_y = y.iloc[val_idx]
    
    model3 = CatBoostClassifier(**params_cb)
    model3.fit(train_x , train_y , eval_set = [(val_x , val_y)], early_stopping_rounds = 3000 , verbose = False)
    preds3 += model3.predict_proba(dft_cb)[:,1]/kf.n_splits
    oof_preds3 += model3.predict_proba(X)[:,1]/kf.n_splits
    roc3.append(accuracy_score(val_y , model3.predict(val_x)))
    fpr, tpr, thresholds = roc_curve(val_y , model3.predict_proba(val_x)[:,1])
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print(n+1 , roc3[n], 'Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
    n+=1

In [ ]:
#Final Threshold
scores = [accuracy_score(y, to_labels(oof_preds3, t)) for t in thresholds]
ix = np.argmax(scores)
print('Threshold=%.3f, accuracy_Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
sub_sample['Survived'] = preds3
sub_sample['Survived'] = sub_sample['Survived'].apply(lambda x: 1 if x>thresholds[ix] else 0)

In [ ]:
sub_sample.to_csv('submission.csv',index=False)